# Fully-Connected Layers Tutorial on Fashion MNIST Data Set 

This is a naive tutorial on how to use `FCLayer` (Fully-connected Layer) to train and predict the Fashion MNIST data set

### First Let's load the Packages

In [1]:
using MLDatasets
using NumNN
using Plots
gr()

Plots.GRBackend()

### Temp for ProgressMeter.jl Package

In [2]:
### uncomment this line the first time you run this code
# ] add https://github.com/timholy/ProgressMeter.jl.git ;

In [3]:
using ProgressMeter
ProgressMeter.ijulia_behavior(:clear);

### Load the Train/Test Data/Labels

In [4]:
X_train, Y_train = FashionMNIST.traindata(Float64);
X_test, Y_test = FashionMNIST.testdata(Float64);

In [5]:
using SigmoidNumbers
P1 = Posit{16,0}
P2 = Posit{16,1}

Base.convert(::Type{P}, b::Bool) where {P <: Posit} = P.(Float64(b))
Posit{N,ES}(b::Bool) where {N,ES} = convert(Posit{N,ES}, b)
Base.exp(a::P) where {P <: Posit} = P.(exp(Float64(a)))
Base.promote_rule(::Type{P}, ::Type{IE}) where {P <: Posit, IE <: Base.IEEEFloat} = P

### Let's Prepare The Data/Labels

In [6]:
X_train ./= 255
X_train = P1.(X_train)
X_test ./= 255
X_test = P1.(X_test)
Y_train = oneHot(Y_train)
Y_test = oneHot(Y_test);

### It's Time fot The Layers 

In [7]:
X_Input = Input(X_train) #or Input(size(X_train))
X = Flatten()(X_Input)
X = FCLayer(120, :relu)(X)
X_Output = FCLayer(10, :softmax)(X);

Another way when there is no side branches is to use the `chain` function as follows:

```julia
X_Input, X_Ouput = chain(X_train,[Flatten(),FCLayer(120,:relu),FCLayer(10,:softmax)]);
```

`chain` returns a `Tuple` of two pointers of the Input `Layer` and Output `Layer`

### Define the Model 

This will also initialize the `Layer`s' parameters

In [8]:
model = Model(X_train,Y_train,X_Input,X_Output, 0.005; optimizer=:adam, paramsDtype=P1);

#### Let's use `predict` to see the current Accuracy

In [9]:
TestP = predict(model, X_test, Y_test);

println()
println("The accuracy of Test Data before the training process $(round(TestP[:accuracy], digits=4))")

Progress: 100%|█████████████████████████████████████████| Time: 0:00:43
  Instances 10000:  10000



The accuracy of Test Data before the training process 0.0926


In [10]:
TrainP = predict(model, X_train, Y_train);

println()
println("The accuracy of Train Data before the training process $(round(TrainP[:accuracy], digits=4))")

Progress: 100%|█████████████████████████████████████████| Time: 0:03:10
  Instances 60000:  60000



The accuracy of Train Data before the training process 0.0968


## Train the model

In [ ]:
TrainD = train(X_train, Y_train, model, 10)#; testData = X_test, testLabels = Y_test);

Progress:  20%|████████▎                                |  ETA: 1:31:13
  Epoch 10:         3
  Instances 60000:  416
  Train Cost:       1.4858
  Train Accuracy:   0.5986

`train` function provides an extra `kwargs` to use test Data/Labels to get the Costs and Accuracies during each training epochs. 

**Note** This will take extra time to do the training

Instead it can be used as follows:

```julia
TrainD = train(X_train, Y_train, model, 10)
```

In [ ]:
plot(1:10, TrainD[:trainAccuracies], label="Training Accuracies")
plot!(1:10, TrainD[:trainCosts], label="Training Costs")
# plot!(1:10, TrainD[:testAccuracies], label="Test Accuracies")
# plot!(1:10, TrainD[:testCosts], label="Test Costs")
ylabel!("Epochs")

### Predict After Training

In [ ]:
TrainP = predict(model, X_train, Y_train);

println()
println("The accuracy of Train Data after the training process $(round(TrainP[:accuracy], digits=4))")

In [ ]:
TestP = predict(model, X_test, Y_test);

println()
println("The accuracy of Test Data after the training process $(round(TestP[:accuracy], digits=4))")